<a href="https://colab.research.google.com/github/ENGCSS/Trabalho_Analise_espacial_de_redes/blob/main/Tb_Redes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# INSTALA AS BIBLIOTECAS NECESSÁRIAS
!pip install osmnx -q                                                         # Instala a biblioteca OSMnx para análise de redes viárias e geodados
!pip install geopandas folium matplotlib fiona -q                                   # Instala as bibliotecas GeoPandas, Folium e Matplotlib
!pip install mapclassify -q                                                   # Instala o Mapclassify para classificação de dados espaciais

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.9 MB/s eta 0:00:00


In [2]:
# IMPORTA AS BIBLIOTECAS NECESSÁRIAS
import osmnx as ox                                                            # Biblioteca para análise de redes viárias e geodados
import folium                                                                 # Biblioteca para criar mapas interativos
import matplotlib.pyplot as plt                                               # Biblioteca para plotagens gráficas
import geopandas as gpd                                                       # Biblioteca para manipulação de dados geoespaciais
import fiona                                                                  # Biblioteca para manipulação de dados geoespaciais
import warnings                                                               # Biblioteca para gerenciar avisos no Python
warnings.filterwarnings("ignore")                                             # Ignorar avisos para manter o console limpo

In [ ]:
# Caminho para o arquivo GeoPackage
geopackage_path = 'caminho/para/seu/arquivo.gpkg'

# Listar todas as camadas no GeoPackage
layers = fiona.listlayers(geopackage_path)

# Exibir as camadas
print(layers)


In [ ]:
# Caminho para o arquivo GeoPackage
geopackage_path = 'caminho/para/seu/arquivo.gpkg'

# Nome da camada que você deseja extrair
layer_name = 'nome_da_camada'

# Ler a camada específica do GeoPackage
gdf = gpd.read_file(geopackage_path, layer=layer_name)

# Exibir as primeiras linhas do GeoDataFrame
print(gdf.head())
